In [1]:
import polars as pl

In [2]:
df_sample = pl.scan_csv("./csv_data/molten_aluminum.csv").head(5).collect()
print("컬럼:", df_sample.columns)
print("\n샘플 데이터:")
print(df_sample)


컬럼: ['time', 'curr', 'currR', 'currS', 'currT', 'Ground', 'PT100', 'Vibra', 'Volt', 'VoltR', 'VoltS', 'VoltT']

샘플 데이터:
shape: (5, 12)
┌─────────────────────────┬───────┬───────┬───────┬───┬───────┬───────┬───────┬───────┐
│ time                    ┆ curr  ┆ currR ┆ currS ┆ … ┆ Volt  ┆ VoltR ┆ VoltS ┆ VoltT │
│ ---                     ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str                     ┆ f64   ┆ f64   ┆ f64   ┆   ┆ f64   ┆ f64   ┆ f64   ┆ f64   │
╞═════════════════════════╪═══════╪═══════╪═══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 2021-04-01T00:00:00.200 ┆ 200.0 ┆ 222.1 ┆ 224.4 ┆ … ┆ 219.4 ┆ 221.0 ┆ 218.4 ┆ 221.9 │
│ 2021-04-01T00:00:00.300 ┆ 300.0 ┆ 222.0 ┆ 224.4 ┆ … ┆ 219.4 ┆ 220.9 ┆ 218.4 ┆ 221.9 │
│ 2021-04-01T00:00:00.400 ┆ 400.0 ┆ 222.0 ┆ 224.5 ┆ … ┆ 219.5 ┆ 220.9 ┆ 218.5 ┆ 221.9 │
│ 2021-04-01T00:00:00.500 ┆ 500.0 ┆ 221.9 ┆ 224.3 ┆ … ┆ 219.4 ┆ 220.8 ┆ 218.4 ┆ 221.8 │
│ 2021-04-01T00:00:00.600 ┆ 600.0 ┆ 221.9 ┆ 224.2 ┆ … ┆ 219.4 ┆ 220.8 ┆ 2

In [3]:
# 2. 시간 기반 split (권장)
df = pl.scan_csv("./csv_data/molten_aluminum.csv")

In [7]:
df

In [4]:
# 시간 파싱 및 정렬
df_sorted = df.with_columns(
    pl.col("time").str.to_datetime("%Y-%m-%dT%H:%M:%S%.3f", strict=False)
).sort("time")

In [5]:
df_sorted

In [5]:
# 시간 파싱 및 정렬
df_sorted = df.with_columns(
    pl.col("time").str.to_datetime("%Y-%m-%d %H:%M:%S%.3f", strict=False)
).sort("time")

In [8]:
print(df_sorted.explain(optimized=True))

SORT BY [col("time")]
   WITH_COLUMNS:
   [col("time").str.strptime(["raise"])] 
    Csv SCAN [./csv_data/molten_aluminum.csv]
    PROJECT */12 COLUMNS
    ESTIMATED ROWS: 4888065


In [9]:
# 정렬된 데이터 샘플 확인
df_sorted_sample = df_sorted.head(5).collect()
print("정렬된 샘플 데이터:")
print(df_sorted_sample)
print(f"\ntime 컬럼 타입: {df_sorted_sample['time'].dtype}")

정렬된 샘플 데이터:
shape: (5, 12)
┌─────────────────────────┬───────┬───────┬───────┬───┬───────┬───────┬───────┬───────┐
│ time                    ┆ curr  ┆ currR ┆ currS ┆ … ┆ Volt  ┆ VoltR ┆ VoltS ┆ VoltT │
│ ---                     ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ datetime[ms]            ┆ f64   ┆ f64   ┆ f64   ┆   ┆ f64   ┆ f64   ┆ f64   ┆ f64   │
╞═════════════════════════╪═══════╪═══════╪═══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 2021-04-01 00:00:00.200 ┆ 200.0 ┆ 222.1 ┆ 224.4 ┆ … ┆ 219.4 ┆ 221.0 ┆ 218.4 ┆ 221.9 │
│ 2021-04-01 00:00:00.300 ┆ 300.0 ┆ 222.0 ┆ 224.4 ┆ … ┆ 219.4 ┆ 220.9 ┆ 218.4 ┆ 221.9 │
│ 2021-04-01 00:00:00.400 ┆ 400.0 ┆ 222.0 ┆ 224.5 ┆ … ┆ 219.5 ┆ 220.9 ┆ 218.5 ┆ 221.9 │
│ 2021-04-01 00:00:00.500 ┆ 500.0 ┆ 221.9 ┆ 224.3 ┆ … ┆ 219.4 ┆ 220.8 ┆ 218.4 ┆ 221.8 │
│ 2021-04-01 00:00:00.600 ┆ 600.0 ┆ 221.9 ┆ 224.2 ┆ … ┆ 219.4 ┆ 220.8 ┆ 218.4 ┆ 221.7 │
└─────────────────────────┴───────┴───────┴───────┴───┴───────┴───────┴───────┴───────┘

time

In [10]:
time_range = df_sorted.select([
    pl.col("time").min().alias("start_time"),
    pl.col("time").max().alias("end_time")
]).collect()

print(f"\n시간 범위: {time_range['start_time'][0]} ~ {time_range['end_time'][0]}")



시간 범위: 2021-04-01 00:00:00.200000 ~ 2021-04-06 16:49:28


In [11]:
total = df_sorted.select(pl.len()).collect().item()

In [15]:
total

4925624

In [30]:
df_collected = df_sorted.collect()

In [27]:
one_df = df_sorted[:int(total / 4)]
two_df = df_sorted[int(total / 4):int(total / 2)]
three_df = df_sorted[int(total / 2):int(total * 3 / 4)]
four_df = df_sorted[int(total * 3 / 4):]

In [31]:
quarter = total // 4
one_df = df_collected.head(quarter)
two_df = df_collected.slice(quarter, quarter)
three_df = df_collected.slice(quarter * 2, quarter)
four_df = df_collected.tail(total - quarter * 3)

In [33]:
print(f"✅ 4등분 Split 완료!")
print(f"1분기: {len(one_df):,} rows ({len(one_df)/total*100:.1f}%)")
print(f"2분기: {len(two_df):,} rows ({len(two_df)/total*100:.1f}%)")
print(f"3분기: {len(three_df):,} rows ({len(three_df)/total*100:.1f}%)")
print(f"4분기: {len(four_df):,} rows ({len(four_df)/total*100:.1f}%)")

✅ 4등분 Split 완료!
1분기: 1,231,406 rows (25.0%)
2분기: 1,231,406 rows (25.0%)
3분기: 1,231,406 rows (25.0%)
4분기: 1,231,406 rows (25.0%)


In [34]:
one_df.write_csv("./csv_data/molten_aluminum_1.csv")
two_df.write_csv("./csv_data/molten_aluminum_2.csv")
three_df.write_csv("./csv_data/molten_aluminum_3.csv")
four_df.write_csv("./csv_data/molten_aluminum_4.csv")

In [ ]:


# 3. Split (70:15:15)
total = df_sorted.select(pl.len()).collect().item()
train_end = int(total * 0.7)
val_end = int(total * 0.85)

df_train = df_sorted.head(train_end).collect()
df_val = df_sorted.slice(train_end, val_end - train_end).collect()
df_test = df_sorted.tail(total - val_end).collect()

# 저장
df_train.write_csv("./csv_data/molten_aluminum_train.csv")
df_val.write_csv("./csv_data/molten_aluminum_val.csv")
df_test.write_csv("./csv_data/molten_aluminum_test.csv")

print(f"\n✅ Split 완료!")
print(f"Train: {len(df_train):,} rows ({len(df_train)/total*100:.1f}%)")
print(f"Val: {len(df_val):,} rows ({len(df_val)/total*100:.1f}%)")
print(f"Test: {len(df_test):,} rows ({len(df_test)/total*100:.1f}%)")


✅ Split 완료!
Train: 3,447,936 rows (70.0%)
Val: 738,844 rows (15.0%)
Test: 738,844 rows (15.0%)
